<div dir="rtl">

<center>

# تشخیص اعداد و یکاهای اندازه‌گیری در متن

آرین احدی نیا، دانا افاضلی، محمد ابول‌نژادیان

<br>

بازیابی پیشرفته اطلاعات

دانشکده مهندسی کامپیوتر، دانشگاه صنعتی شریف

</center>

</div>

<div dir="rtl">

## مقدمه

در این نوتبوک قصد داریم مسئله‌ی تشخیص واحدهای اندازه گیری را و همچنین مسئله تبدیل واحد را حل کنیم.

ساختار پروژه به شکل زیر است:

در کنار این نوتبوک ۲ پوشه قرار دارند که در پوشه ```resources``` فایل‌هایی که به عنوان دیتا یا مدل از قبل آماده شده هستند قرار دارند. مانند مدل 
```POSTagger```
برای هضم و همچنین دیتای گرفته شده از سایت باحساب برای برخی از واحدها.


همچنین در پوشه 
```src```
، تمامی کدهای زده شده قرار دارد که به دلیل حجم زیاد و اینکه عملا در لایه نهایی استفاده نمیشوند از نوتبوک حذف شده اند. در ادامه و در قسمت مربوطه به توضیح آنها پرداخته میشود.

در انتهای هر بخش تعدادی تست کیس نمونه قرار داده شده تا فانکشنالیتی آن قسمت برسی شود.

در بخش اول قسمت اصلی یعنی پیدا کردن واحدها برسی میشود و در قسمت بعد تبدیل واحد برسی میشود.
</div>

<div dir="rtl">

## وابستگی‌ها

</div>

In [3]:
# nothing fancy
%pip install hazm
%pip install pandas

     |████████████████████████████████| 316 kB 31.4 MB/s            
     |████████████████████████████████| 1.4 MB 63.8 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 233 kB 53.8 MB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394468 sha256=69670e5104078840d9c51b74b4152d5b1656b3f3a0f6b55e3e6c578f659d71eb
  Stored in directory: /home/datalore/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl size=193066 sha256=7b5e1bbc3ce994f54acdc58076ac996e2e848b15ee11c01b234d25ba00e8a81c
  Stored in directory: /home/datalore/.cache/pip/wheels/3c/d8/9f/59fd78b2b7d1e9ffcb68fb6de80c2e7c20b804c8cbc4d8fc23
Successfully built nltk libwapiti
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
Note: you 

In [5]:
from src.pipelines import *
from src.base import SpeechTagsPipeline
from src.utils import filter_non_units, convert, match_patterns

<div dir="rtl">

## تعریف مساله

تعریف مسئله به این شکل است: یک استرینگ به عنوان ورودی به تابع
```run(org_str)```
داده میشود و خروجی آن مواردی از واحدها، عددهای مربوطه، مرجع عدد و واحد پیدا شده (در صورت وجود) و همچنین بازه این موارد در استرینگ میباشد.
</div>

<div dir="rtl">

## روند کلی

پردازش متن برای بدست آمدن نتیجه مورد نظر از دو فاز تشکیل شده است.

1. یک خط لوله برای برچسب گذاری اعداد، واحدها، اسم‌ها و صفت‌ها.
2. تطبیق برچسب‌های بدست آمده با الگوهای زبانی عدد و یکا.

</div>

<div dir="rtl">

## فاز اول: برچسب‌گذاری

با توجه به نیاز مساله، برچسب هایی از قبیل عدد، اسم، صفت ، کمیت، واحد و تعدادی برچسب کمکی تعریف می‌کنیم و در یک خط لوله تلاش می‌کنیم که این برچسب‌ها را با یک ترتیب منطقی پیدا کنیم. توجه بفرمایید که این برچسب‌ها هر یک در بازه‌ای قرار می‌گیرند بنابرین در نهایت با تجمیع این بازه‌های می‌توانیم بازه نهایی مورد نظر را بدست آوریم.

</div>

<div dir="rtl">

### خط لوله برچسب گذاری

در یک خط لوله، به ترتیب منطقی ابتدا برچسب‌ها را با توجه به برچسب‌های قبلی بدست آمده، بدست می‌آوریم و در نهایت، تمام برچسب‌های سخن را برای فاز دوم بر‌می‌گردانیم.

ترتیب پیدا کردن برچسب‌ها به شکل زیر میباشد: ابتدا اعداد را پیدا میکنیم، سپس مراحل نسبتا زیادی را صرف پیدا کردن و تجزیه و تحلیل واحدها و کمیت‌های داخل متن میکنیم و در نهایت برچسب های مربوط به اسامی و صفت‌ها را پیدا میکنیم.

این قسمت به شکل یک پایپلاین پیاده سازی شده که در هر مرحله، یک پایپ استرینگ اولیه را به همراه تمامی برچسب‌های پیدا شده تحت یک لیست دریافت میکند. سپس با پردازش استرینگ ورودی، به لیست برچسب‌های پیدا شده، برچسب‌های خود را اضافه میکند. شمای یک برچسب در حالت کلی به شکل زیر میباشد:

</div>

src.base.SpeechTag:

```
class SpeechTag(object):

    def __init__(self, tag: Tag, span: Span, value: Union[int, float, str, None]):
        self.tag = tag
        self.span = span
        self.value = value

    @staticmethod
    def to_pattern_str(st_list):
        return ''.join(map(lambda st: st.tag.value, st_list))
```

<div dir="rtl">

همچنین شمای یک پایپ به صورت کلی به شکل زیر میباشد:

</div>

src.pipelines.get_xxx_tags_pipe:

```
# Note the general prototpye for the pipe
def get_number_tags_pipe(org_str: str, speech_tags: List[SpeechTag]) -> List[SpeechTag]:
    # Some processing...
    scientific_nums = number_extractor(org_str)
    speech_tags = speech_tags + scientific_nums

    # Create newly found speech tags and add them to the main list upon validation
    for num in default_number_extractor.run(org_str):
        num_span = Span(num['span'][0], num['span'][1])
        if not any([x.span.is_overlapped(num_span) for x in scientific_nums]):
            speech_tags.append(NumberTag(num_span, num['value']))
        
    # Finally return the cumulated list
    return speech_tags
```

<div dir="rtl">

### اعداد

برای پیدا کردن اعداد از دو روش همزمان استفاده میشود. نخست با استفاده از رجکس اعدادی که فرمت عددی دارند پیدا شده، و سپس با استفاده ابزار parsi.io به استخراج اعداد با فرمت متنی میپردازیم. در ادامه تست این تابع را مشاهده میکنید:
</div>

In [5]:
# Just to Show
[
    get_number_tags_pipe('دو کیلو و نیم', []),
    get_number_tags_pipe('دو کیلو و صد گرم', []),
    get_number_tags_pipe('یک متر و بیست سانت', []),
    get_number_tags_pipe('هشتاد و پنج صدم وات', []),
    get_number_tags_pipe('.21x10**50', []),
    get_number_tags_pipe('120.1', []),
    get_number_tags_pipe('120.', []),
    get_number_tags_pipe('1.022*10^23', []),
    get_number_tags_pipe('1.022', []),
    get_number_tags_pipe('1e3', []),
    get_number_tags_pipe('-18732.787e-100', [])
]

[[<NUM 2 in (0, 2)>, <NUM 0.5 in (10, 13)>],
 [<NUM 2 in (0, 2)>, <NUM 100 in (10, 12)>],
 [<NUM 1 in (0, 2)>, <NUM 20 in (9, 13)>],
 [<NUM 80.05 in (0, 15)>],
 [<NUM 2.1e+49 in (0, 10)>],
 [<NUM 120.1 in (0, 5)>],
 [<NUM 120 in (0, 3)>],
 [<NUM 1.0219999999999999e+23 in (0, 11)>],
 [<NUM 1.022 in (0, 5)>],
 [<NUM 1000.0 in (0, 3)>],
 [<NUM -1.8732787e-96 in (0, 15)>]]

<div dir="rtl">

### یکاها

</div>

<div dir="rtl">

واحدها به طرق مختلف با یک دیگر ترکیب می‌شوند و واحدهای جدید را بوجود می‌آورند. به عنوان مثال اضافه شدن یک پیشوند به یک واحد ساده، یک واحد پیشوندی بوجود می‌آورد (به عنوان مثال: کیلومتر = کیلو + متر) یا اضافه شدن یک مرتبه‌ساز (مربع، مکعب، مجذور) با یک واحد ساده یا یک واحد پیشوندی، یک واحد مرتبه‌دار بوجود می‌آورد (به عنوان مثال: کیلومتر مربع: کیلومتر + مربع).

عبارت‌های پایه‌ای سازنده واحدها عبارتند از:
<ul>
    <li> • واحدهای ساده (Su) مانند گرم، نیوتن، پاسکال، متر و ... </li>
    <li> • پیشوندها (Sip) مانند کیلو، میلی، مگا و ... </li>
    <li> • مرکب‌سازها (Cmp) مانند "بر"، "در" و ... </li>
</ul>

حال میتوانیم علاوه بر واحدهای ساده، انواعی از واحدها را داشته باشیم:

<ul>
    <li> • واحدهای پیشوندی (Pu) </li>
    <li> • واحدهای مرتبه‌دار (Ou) </li>
    <li> • واحدهای مرکب (U) </li>
</ul>
با توجه به اینکه نحو ترکیب واحدها کمابیش مشخص است، یک دستور نحوی میتوانیم برای آن تعریف کنیم. چنین تعریفی به طریق زیر خواهد بود.


</div>

<div dir="ltr">
<center>

$U::=Ou|UOu|UCmpOu$

$Ou::=Pu|OrdmPu|PuOrdm$

$Pu::=Su|SipSu$

$Su$ is defined by a set of units which we get in dataset.

<small>

$Su$: Simple Unit,
$Sip$: SI Prefix,
$Pu$: Prefixed Unit,
$Ordm$: Order Modifier,
$Cmp$: Compounder,
$U$: Unit

</small>

</center>
</div>


<div dir="rtl">

با توجه به دستور فوق، میتوان درخت نحوی یک واحد بسیار پیچیده را ایجاد کرد. درخت زیر را در نظر بگیرید که واحد "کیلوگرم میلی‌متر مربع بر مجذور نانوثانیه" را توصیف میکند.

<img src="resources/unit_tree.png">


در این پیاده‌سازی، ما درخت نحوی واحدها را می‌سازیم. همچنین در داده اولیه در کنار پیشوند‌ها ضریب آنها را و در کنار واحدها نسبت آنها به واحد SI را ذخیره می‌کنیم. همچنین رفتار مرتبه‌سازها و مرکب‌سازها را که با واحدشان دارند را در قالب یک تابع در کنار هر یک ذخیره می‌کنیم. به عنوان مثال مرکب‌ساز "بر" نسبت‌ها را بر هم تقسیم می‌کند و مرتبه‌ساز مربع نسبت مورد نظر را به توان دو می‌رساند. به این طریق با محاسبه بازگشتی، میتوانیم نسبت اندازه هر واحد مرکب به واحد SI مربوطه را بدست آوریم.

همچنین برای بدست آمدن کمیت مربوط به هر واحد، آن واحد را تحلیل ابعادی میکنیم. به عنوان مثال واحدهای نیرو در ابعاد  قرار دارند. برای این منظور ابعاد کمیت‌های مختلف را در داده اولیه قرار می‌دهیم و با توجه به آن مشابه ضریب تبدیل به SI، ابعاد واحدهای مرکب را محاسبه می‌کنیم و آن را با کمیت‌های مختلف تطبیق می‌دهیم.

</dir>

In [6]:
test_unit_pipeline = SpeechTagsPipeline([
    get_number_tags_pipe,
    get_quantity_name_tags_pipe,
    get_simple_unit_tags_on_names_pipe,
    get_simple_unit_tags_on_symbols_pipe,
    get_si_prefix_tags_on_names_pipe,
    get_si_prefix_tags_on_symbols_pipe,
    get_order_modifier_tags_pipe,
    get_unit_compounder_preposition_tags_pipe,
    speech_tags_sorter_pipe,
    get_prefixed_units_pipe,
    get_ordered_units_pipe,
    get_units_pipe
])

In [12]:
# Just to Show
[
    test_unit_pipeline.run('دو نیوتن بر متر مربع و نیم'),
    test_unit_pipeline.run('دو کیلومتر و صد گرم'),
    test_unit_pipeline.run('یک متر و بیست سانت'),
    test_unit_pipeline.run('هشتاد و پنج صدم وات'),
]

[[<NUM 2 in (0, 2)>,
  <Unit <Unit <OrdUnit None <PfxUnit None <SUnit نیوتن at (3, 8)> at (3, 8)> at (3, 8)> None None> <Cmp بر at (9, 11)> <OrdUnit <OMfr مربع at (16, 20) <PfxUnit None <SUnit متر at (12, 15)> at (12, 15)> at (12, 20)>>,
  <NUM 0.5 in (23, 26)>],
 [<NUM 2 in (0, 2)>,
  <Unit <OrdUnit None <PfxUnit <Pfx کیلو at (3, 7)> <SUnit متر at (7, 10)> at (3, 10)> at (3, 10)> None None>,
  <NUM 100 in (13, 15)>,
  <Unit <OrdUnit None <PfxUnit None <SUnit گرم at (16, 19)> at (16, 19)> at (16, 19)> None None>],
 [<NUM 1 in (0, 2)>,
  <Unit <OrdUnit None <PfxUnit None <SUnit متر at (3, 6)> at (3, 6)> at (3, 6)> None None>,
  <NUM 20 in (9, 13)>],
 [<NUM 80.05 in (0, 15)>,
  <Unit <OrdUnit None <PfxUnit None <SUnit وات at (16, 19)> at (16, 19)> at (16, 19)> None None>]]

<div dir="rtl">
در هر مثال میتوانید واحدهایی که پیدا شده اند را به حالت درختی آنها، یعنی پیشوند، واحد، مرکب ساز و ... مشاهده کنید.
</div>

<div dir="rtl">

### نقش‌های زبانی

در نهایت بعد از پردازش واحدها و یکاها، اسامی و صفت‌ها را برسی میکنیم. برای این منظور از یک مدل 
```POSTagger```
استفاده میکنیم که در مدل آنرا از سایت این درس برداشته‌ایم. در این قسمت در واقع تمام مواردی که میتوانند 
```item```
باشند را به عنوان اسم گرفته ایم. همچنین برای پیدا کردن توصیفات کمیتی
(مانند سرعت زیاد که باید در جواب نهایی در نظر گرفته شود) صفت‌ها را نیز پیدا کرده‌ایم.

</div>

In [20]:
# Just to Show
[
    [(st.tag, st.span.to_tuple(), st.value) for st in get_postags_pipe('این یک تست عجیب و زیبا میباشد', [])],
    [(st.tag, st.span.to_tuple(), st.value) for st in get_postags_pipe('هیچگاه انقدر خفن خوابم نمیامده', [])],
    [(st.tag, st.span.to_tuple(), st.value) for st in get_postags_pipe('و چه کارهای خطیری که بخاطر فرار نمیکنیم', [])],
    [(st.tag, st.span.to_tuple(), st.value) for st in get_postags_pipe('و قسم به ددلاین', [])],
]

[[(<Tag.NOUN: 'N'>, (0, 3), 'این'),
  (<Tag.NOUN: 'N'>, (7, 10), 'تست'),
  (<Tag.ADJECTIVE: 'J'>, (11, 15), 'عجیب'),
  (<Tag.ADJECTIVE: 'J'>, (18, 22), 'زیبا')],
 [(<Tag.NOUN: 'N'>, (13, 16), 'خفن'), (<Tag.NOUN: 'N'>, (17, 22), 'خوابم')],
 [(<Tag.NOUN: 'N'>, (5, 11), 'کارهای'),
  (<Tag.ADJECTIVE: 'J'>, (12, 17), 'خطیری'),
  (<Tag.NOUN: 'N'>, (27, 31), 'فرار')],
 [(<Tag.NOUN: 'N'>, (2, 5), 'قسم'), (<Tag.NOUN: 'N'>, (9, 15), 'ددلاین')]]

<div dir="rtl">
همانطور که میبینید مدل تگر میتوانست بهتر باشد...
</div>

<div dir="rtl">

## فاز دوم: تطبیق با الگوها


در این قسمت، توابع جواب قسمت‌ قبل را گرفته و با استفاده از پترن‌های هارد کد شده سعی میکنند که خواسته‌های مسئله را استخراج کنند. پترن ها به طور کلی به شکل زیر هستند:


</div>

```
patterns = [
    'D+\s*U+\s*N+',  # علی ۳۰۰ کیلو شیشه خرید (also handles 200kg) 1
    'D+\s*U+',  # اتوبان ۲۰۰ دقیقه تا فلان فاصله دارد  2
    'Q+\s*N+\s*J+\s*D+\s*U+',  # وزن علی کوچولو ۲۰۰ کیلو است 3
    'Q+\s*N+\s*D+\s*U+',  # وزن علی ۲۰۰ کیلو است 4
    'N+\s*J+\s*D+\s*U+',  # علی کوچولو ۲۰۰ کیلو است 5
    'Q+\s*J+',  # سرعت زیاد 7
    'N+\s*D+\s*U+\s*Q+',  # شیشه ۲۰۰ کیلو وزن دارد 8
    'Q+\s*D+\s*U+',  # به وزن ۲۰۰ کیلو 9
    'D+\s*U+\s*.\s*D+\s*U+',
    'D+\s*U+\s*.\s*D+\s*U+\s*N+',  # ۲ متر و ۳۰ سانت پارچه خرید 11
]
```

In [21]:
# complete pipeline for final test

extractor_pipeline = SpeechTagsPipeline([
    get_number_tags_pipe,
    get_quantity_name_tags_pipe,
    get_simple_unit_tags_on_names_pipe,
    get_simple_unit_tags_on_symbols_pipe,
    get_si_prefix_tags_on_names_pipe,
    get_si_prefix_tags_on_symbols_pipe,
    get_order_modifier_tags_pipe,
    get_unit_compounder_preposition_tags_pipe,
    speech_tags_sorter_pipe,
    get_prefixed_units_pipe,
    get_ordered_units_pipe,
    get_units_pipe,
    filter_pipe,
    get_postags_pipe,
])

<div dir="rtl">

در نهایت به تابع نهایی میرسیم. که همانطور که خواسته شده بود تنها یک استرینگ به عنوان ورودی دریافت میکند. اول از همه برچست ‌های مربوط به استرینگ را پیدا میکند و سپس با استفاده از توابع نوشته شده، پترن‌ها را استخراج و مچ میکند. در نهایت تمامی آیتم‌های ساخته شده را خروجی میدهد.

</div>

In [23]:
def run(org_str):
    speech_tags = extractor_pipeline.run(org_str)
    # print(speech_tags)
    created_objects_output = match_patterns(org_str, speech_tags)
    return created_objects_output

In [34]:
# Just to Show
test_list = [
    [print(o) for o in run('علی ۴۰ کیلو وزن دارد و بار به این سنگین است.')],
    [print(o) for o in run('تندی زیاد نیست')],
    [print(o) for o in run('علی ۲ kg شیشه خرید.')]
]

{'type': 'جرم', 'amount': 40, 'unit': 'کیلو', 'item': 'علی', 'marker': 'علی ۴۰ کیلو وزن', 'span': (0, 15)}
{'type': 'سرعت', 'amount': '', 'unit': '', 'item': '', 'marker': 'تندی زیاد', 'span': (0, 9)}
{'type': 'جرم', 'amount': 2, 'unit': 'kg', 'item': '', 'marker': '۲ kg', 'span': (4, 8)}


<div dir="rtl">

## تست روی داده ها و دیدن نتایج


در این قسمت تعدادی تست کیس آماده شده که تست و نمایش داده میشود. این تست کیس‌ها را میتوانید در
```resources/unit_tests.txt```
مشاهده کنید
</div>

In [31]:
tests_file = open("resources/unit_tests.txt", "rt")
tests = tests_file.read().split("\n")
tests_file.close()
for test in tests:
    created_objects = run(test)
    for unit_object in created_objects:
        print(unit_object)

علی با سرعت زیاد به سمت خانه آمد. او میدانست که باید ۵ مترمربع پارچه بخرد و آن را با سرعت 5km/s به خانه برساند که این فشار زیاد به او میخوراند. الان که دارم این حرف را میزنم احتمالا ۵ ساعت و ۴ دقیقه طول بکشد تا تو آن را بخوانی، ولی بدان که اگر با فشار ۴ اتمسفرر هوا را با توان ۵ کیلووات به سمت من هل دهی،َ باز هم من مقاومت میکنم و عصای پنج متر و شش سانتی متری ام را حلقت فرو میکنم.
{'type': 'سرعت', 'amount': '', 'unit': '', 'item': '', 'marker': 'سرعت زیاد', 'span': (7, 16)}
{'type': 'مساحت', 'amount': 5, 'unit': 'مترمربع', 'item': 'پارچه', 'marker': '۵ مترمربع پارچه', 'span': (53, 68)}
{'type': 'سرعت', 'amount': 5, 'unit': 'km/s', 'item': '', 'marker': 'سرعت 5km/s', 'span': (85, 95)}
{'type': 'فشار', 'amount': '', 'unit': '', 'item': '', 'marker': 'فشار زیاد', 'span': (118, 127)}
{'type': 'زمان', 'amount': 5, 'unit': 'ساعت', 'item': '', 'marker': '۵ ساعت', 'span': (182, 188)}
{'type': 'زمان', 'amount': 4, 'unit': 'دقیقه', 'item': '', 'marker': '۴ دقیقه', 'span': (191, 198)}
{'type': 'فشا

<div dir="rtl">

# بخش امتیازی: تبدیل واحد

با توجه به درختی که در قسمت قبل ساخته شده، میتوان به راحتی تبدیل واحد را نیز انجام داد. در این قسمت این قسمت را پیاده سازی و تست میکنیم. تابع نهایی به صورت
```run_conver(value, source, destination)```
است که آرگمان‌ها به ترتیب بیانگر مقدار برای تبدیل، واحد مبدا و واحد مقصد میباشند که دو ورودی آخر از جنس استرینگ میباشند. فرض شده که دو واحد ورودی به هم قابل تبدیل هستند.

این تابع نیز از دو بخش تشکیل شده. نخست واحدها تجزیه و تحلیل میشوند، سپس با تبدیل آنها به واحد
SI
تبدیل آنها صورت میگیرد.

</div>

In [32]:
# pipeline for analyzing units
unit_determiner_pipeline = SpeechTagsPipeline([
    get_number_tags_pipe,
    get_quantity_name_tags_pipe,
    get_simple_unit_tags_on_names_pipe,
    get_simple_unit_tags_on_symbols_pipe,
    get_si_prefix_tags_on_names_pipe,
    get_si_prefix_tags_on_symbols_pipe,
    get_order_modifier_tags_pipe,
    get_unit_compounder_preposition_tags_pipe,
    speech_tags_sorter_pipe,
    get_prefixed_units_pipe,
    get_ordered_units_pipe,
    get_units_pipe,
    filter_non_units,
])

In [33]:
def run_convert(val, source, destination):
    src_unit = unit_determiner_pipeline.run(source)[0]
    dst_unit = unit_determiner_pipeline.run(destination)[0]
    return convert(val, src_unit, dst_unit)

In [37]:
# Just to Show
[
    run_convert(1, 'کیلوگرم',
                'پیکوگرم'),
    run_convert(1, 'متر مربع',
                'هکتار'),
    run_convert(1, 'اتمسفر',
                'پاسکال'),
    run_convert(1, 'سال نوری',
                'متر'),
]

[1000000000000000.0, 0.0001, 101325.0, 9460000000000000.0]